In [ ]:
from seqnado.config import WorkflowConfig
from seqnado.config import (
    GenomeConfig,
    ProjectConfig,
    BowtieIndex,
    AssayConfig,
    BigwigConfig,
    PeakCallingConfig,
    SpikeInConfig,
    UCSCHubConfig,
    PlottingConfig,
    WorkflowConfig
)
from seqnado import Assay
from datetime import datetime
from pathlib import Path
from jinja2 import Template, Environment, FileSystemLoader
from seqnado.config.user_input import get_tool_options

In [ ]:
wc = WorkflowConfig(
    assay=Assay.RNA,
    genome=GenomeConfig(
        name="GRCh38",
        index=BowtieIndex(
            prefix='../../../reference/hg38_ecoli_b2t_index/bt2_index/hg38_ecoli'
        )
    ),
    project=ProjectConfig(
        name="Test Project",
        date=datetime.today(),
        description="This is a test project",
        output_dir="test_output",
        directory="test_directory",
    ),
    metadata=Path("test_metadata.yaml"),
    assay_config=AssayConfig.RNA.value(
        bigwigs=BigwigConfig(
            pileup_method=
    )
    options=get_tool_options(Assay.RNA)
)



SyntaxError: expected argument value expression (2096520468.py, line 18)

In [ ]:
ASSAY_CONFIG_MAP[Assay.RNA].model_fields

{'bigwigs': FieldInfo(annotation=Union[BigwigConfig, NoneType], required=False, default=None),
 'plotting': FieldInfo(annotation=Union[PlottingConfig, NoneType], required=False, default=None),
 'ucsc_hub': FieldInfo(annotation=Union[UCSCHubConfig, NoneType], required=False, default=None),
 'dataset_for_ml': FieldInfo(annotation=Union[MLDatasetConfig, NoneType], required=False, default=None),
 'create_geo_submission_files': FieldInfo(annotation=bool, required=False, default=False),
 'create_heatmaps': FieldInfo(annotation=bool, required=False, default=False),
 'options': FieldInfo(annotation=str, required=False, default=''),
 'rna_quantification': FieldInfo(annotation=Union[RNAQuantificationConfig, NoneType], required=False, default=None)}

In [ ]:
dumped = wc.model_dump_json()

In [ ]:
WorkflowConfig.model_validate_json(dumped)

WorkflowConfig(assay=<Assay.RNA: 'rna'>, project=ProjectConfig(name='Test Project', date=datetime.datetime(2025, 7, 27, 18, 46, 49, 632023), directory=PosixPath('test_directory')), genome=GenomeConfig(name='GRCh38', index=BowtieIndex(type='Bowtie2', prefix='../../../reference/hg38_ecoli_b2t_index/bt2_index/hg38_ecoli'), fasta=None, chromosome_sizes=None, gtf=None, genes=None, blacklist=None, organism=None, version=None), metadata=PosixPath('test_metadata.yaml'), pcr_duplicates=PCRDuplicatesConfig(strategy=<PCRDuplicateHandling.NONE: 'none'>, tool=<PCRDuplicateTool.PICARD: 'picard'>), qc=QCConfig(library_size=True, fraction_of_reads_in_peaks=True), assay_config=None, options='bamnado:\n  bamcoverage: --bin-size 10 --norm-method cpm\n  threads: 8\ndeeptools:\n  alignmentsieve: --minMappingQuality 30\n  bamcoverage: -bs 1 --normalizeUsing CPM\n  threads: 16\nfeaturecounts:\n  options: -s 0 -p --countReadPairs -t exon -g gene_id\n  threads: 16\nheatmap:\n  colormap: RdYlBu_r\n  options: -b

In [ ]:
WorkflowConfig.model_fields

{'assay': FieldInfo(annotation=Assay, required=True),
 'project': FieldInfo(annotation=ProjectConfig, required=True),
 'genome': FieldInfo(annotation=GenomeConfig, required=True),
 'metadata': FieldInfo(annotation=Path, required=True),
 'pcr_duplicates': FieldInfo(annotation=PCRDuplicatesConfig, required=False, default=PCRDuplicatesConfig(strategy=<PCRDuplicateHandling.NONE: 'none'>, tool=<PCRDuplicateTool.PICARD: 'picard'>)),
 'qc': FieldInfo(annotation=QCConfig, required=False, default=QCConfig(library_size=True, fraction_of_reads_in_peaks=True)),
 'assay_config': FieldInfo(annotation=Union[ATACAssayConfig, ChIPAssayConfig, CATAssayConfig, RNAAssayConfig, SNPAssayConfig, MCCAssayConfig, METHAssayConfig, CRISPRAssayConfig, NoneType], required=False, default=None),
 'options': FieldInfo(annotation=Union[str, NoneType], required=False, default=None)}

In [ ]:
dumped

'{"assay":"rna","project":{"name":"Test Project","date":"2025-07-27T18:46:49.632023","directory":"test_directory"},"genome":{"name":"GRCh38","index":{"type":"Bowtie2","prefix":"../../../reference/hg38_ecoli_b2t_index/bt2_index/hg38_ecoli"},"fasta":null,"chromosome_sizes":null,"gtf":null,"genes":null,"blacklist":null,"organism":null,"version":null},"metadata":"test_metadata.yaml","pcr_duplicates":{"strategy":"none","tool":"picard"},"qc":{"library_size":true,"fraction_of_reads_in_peaks":true},"assay_config":null,"options":"bamnado:\\n  bamcoverage: --bin-size 10 --norm-method cpm\\n  threads: 8\\ndeeptools:\\n  alignmentsieve: --minMappingQuality 30\\n  bamcoverage: -bs 1 --normalizeUsing CPM\\n  threads: 16\\nfeaturecounts:\\n  options: -s 0 -p --countReadPairs -t exon -g gene_id\\n  threads: 16\\nheatmap:\\n  colormap: RdYlBu_r\\n  options: -b 1000 -m 5000 -a 1000 --binSize 50\\nhomer:\\n  makebigwig: null\\n  maketagdirectory: null\\npicard:\\n  options: null\\n  threads: 8\\nsalmon:\

In [ ]:
env = Environment(loader=FileSystemLoader("."))
template = env.get_template("template.jinja")

In [ ]:
with open("output.yml", "w") as f:
    r = wc.model_dump(mode="json", exclude_none=True)
    r = template.render(**r)
    f.write(r)
